<a href="https://colab.research.google.com/github/yj5x/AI-tasks/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#تحليل بيانات العملاء في متجر إلكتروني

##الهدف:

##تحليل بيانات العملاء من متجر إلكتروني باستخدام تقنيات

##Unsupervised Learning
##(التعلم غير الخاضع للإشراف)


##لفهم أنماط الشراء وسلوك المستخدمين، واقتراح استراتيجيات لتحسين التسويق

## 1. تحميل المكتبات
- استيراد المكتبات الأساسية مثل `pandas`, `numpy`, `matplotlib`, `seaborn`

## 2. تحميل البيانات
- ربط جوجل كولاب
- قراءة ملف CSV وحفظه في DataFrame




In [ ]:
# ============================
#           المكتبات
# ============================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ============================
#         تحميل البيانات
# ============================

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/AI in business /CC_GENERAL.csv')
df = data.copy()


## 3. استكشاف البيانات (EDA)
- عرض شكل البيانات (صفوف وأعمدة)
- عرض عينة من البيانات
- فحص أنواع البيانات والقيم المفقودة
- حذف الأعمدة غير الضرورية
- تعبئة القيم المفقودة

In [ ]:

# ============================
#       استكشاف البيانات
# ============================

# عدد الصفوف والأعمدة
display("Number of rows and columns", df.shape)
print("\n")

# أول 5 صفوف من البيانات
display(df.head())
print("\n")

# أنواع البيانات والقيم المفقودة
df.info()
display(df.isnull().sum())
print("\n")


In [ ]:

# نظرة على محتوى كل عمود
for col in df.columns:
    display(f"{col}: {df[col].unique()[:5],}")
    display('='*100)

# حذف البيانات غير المفيدة
df.drop('CUST_ID', axis=1, inplace=True)

# تعبئة القيم الناقصة
df.fillna(df.mean(), inplace=True)

# لعرض القيم المفقودة إن وُجدت
display(df.isnull().sum())



## 4. تحجيم البيانات (Scaling)
- توحيد مقاييس البيانات باستخدام `StandardScaler

In [ ]:
# ============================
#         Scaling
# ============================

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

df_scaled.head()

### 5. التحليل العنقودي الهرمي (Dendrogram)
- إنشاء Dendrogram


- لتحديد عدد الاعناقيد

In [ ]:
# ============================
#        Dendrogram
# ============================

from scipy.cluster.hierarchy import linkage, dendrogram

Z = linkage(df_scaled, 'ward')


plt.figure(figsize=(10, 4))
dendrogram(Z, truncate_mode='lastp', p=12, show_leaf_counts=True, leaf_font_size=8)
plt.title('Dendrogram')
plt.xlabel('customers')
plt.ylabel('distance')
plt.axhline(y=30, color='r', linestyle='--')
plt.show()



## 6. طريقة الكوع (Elbow Method)

- تحديد العدد الأمثل للعناقيد باستخدام K-Means

In [ ]:

# ============================
#   Elbow Method for KMeans
# ============================

from sklearn.cluster import KMeans

inertia = []

# افتراض قيمه k
for k in range(1,50):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

#  هنا حشوف عالرسمه متى يبدء المنحنى يستقر واحد قيمه K
plt.plot( inertia, marker='o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


## 7. التحليل العنقودي بـ K-Means
- تطبيق خوارزمية K-Means
- تصور النتائج باستخدام PCA
- تقييم النموذج باستخدام Silhouette Score
- مقارنة متوسطات القيم لكل عنقود
- إنشاء رسوم بيانية للسلوك العام لكل مجموعة

In [ ]:
K = 12
# ============================
#     K-Means Clustering
# ============================

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
clusters = kmeans.fit_predict(df_scaled)

df_scaled['Cluster_KMeans'] = clusters

#
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_scaled.drop('Cluster_KMeans', axis=1))


#
plt.figure(figsize=(8, 6))
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=clusters, cmap='viridis', s=50)
plt.title('K-Means Clustering (with PCA)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.grid(True)
plt.colorbar(label='Cluster ID')
plt.show()


# ============================
#       تقييم - KMeans
# ============================

score = silhouette_score(df_scaled.drop('Cluster_KMeans', axis=1), clusters)
print(f'Silhouette Score for K-Means: {score:.3f}')



# ============================
#           التباين
# ============================

explained_var = pca.explained_variance_ratio_
print(f"Explained Variance Ratio: {np.round(explained_var,2)}")
print(f"cumulative_variance: {np.round(np.cumsum(explained_var),2)}")
print(f"Total Variance Retained: {np.round(np.sum(explained_var)*100,2)}%")

In [ ]:
# ============================
#  ربط العناقيد بالبيانات
# ============================

df['Cluster'] = clusters


# ==================================
#  عرض بيانات كل عنقود او مجموعه
# ==================================
cluster_profiles = df.groupby('Cluster').mean()
display(cluster_profiles)




In [ ]:
# ============================
#     الرسم البياني للنتائج
# ============================

#
clustered_df = df.copy()
cluster_means = clustered_df.groupby('Cluster').mean()

cluster_means[['BALANCE', 'PURCHASES', 'PAYMENTS']].plot(kind='bar', figsize=(10, 6))
plt.title('Average Behavior per Cluster')
plt.ylabel('Amount')
plt.xlabel('Cluster')
plt.grid(True)
plt.show()

## 8. التحليل العنقودي بستخدامDBSCAN
- تطبيق خوارزمية DBSCAN
- تصور النتائج ثنائية وثلاثية الأبعاد
- تقييم النموذج
- مقارنة متوسطات القيم لكل عنقود
- إنشاء رسوم بيانية للسلوك العام لكل مجموعة


In [ ]:


# ============================
#     اختيار eps - DBSCAN
# ============================

from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=5)
nbrs = neigh.fit(df_scaled)
distances, indices = nbrs.kneighbors(df_scaled)

distances = np.sort(distances[:, 4], axis=0)
plt.plot(distances)
plt.title('k-distance Graph to Determine eps')
plt.xlabel('Points sorted by distance')
plt.ylabel('4th Nearest Neighbor Distance')
plt.grid(True)
plt.show()





In [ ]:

# ============================
#           DBSCAN
# ============================

from sklearn.cluster import DBSCAN


dbscan = DBSCAN(eps=2, min_samples=5)
dbscan_labels = dbscan.fit_predict(df_scaled)

# رسمه PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_scaled)

df['PCA1'] = pca_data[:, 0]
df['PCA2'] = pca_data[:, 1]
df['Cluster_DBSCAN'] = dbscan_labels

plt.figure(figsize=(7, 4))
sns.scatterplot(data=df, x='PCA1', y='PCA2', hue='Cluster', palette='Set1')
plt.title('DBSCAN Clusters (PCA Reduced)')
plt.grid(True)
plt.show()


# ============================
#   تقييم DBSCAN (Silhouette)
# ============================

mask = df['Cluster_DBSCAN'] != -1
score = silhouette_score(df_scaled[mask], df['Cluster_DBSCAN'][mask])
print(f"Silhouette Score for DBSCAN: {score:.3f}")



# ============================
#           التباين
# ============================

explained_var = pca.explained_variance_ratio_
display(f"Explained Variance Ratio: {np.round(explained_var,2)}")
display(f"cumulative_variance: {np.round(np.cumsum(explained_var),2)}")
display(f"Total Variance Retained: {np.round(np.sum(explained_var)*100,2)}%")

In [ ]:
# ============================
#      بيانات كل مجموعة
# ============================
cluster_summary = df.groupby('Cluster_DBSCAN').mean()
display(cluster_summary)


In [ ]:
# ============================
#     الرسم البياني للنتائج
# ============================

clustered_df = df.copy()
cluster_means = clustered_df.groupby('Cluster_DBSCAN').mean()

cluster_means[['BALANCE', 'PURCHASES', 'PAYMENTS']].plot(kind='bar', figsize=(10, 6))
plt.title('Average Behavior per Cluster')
plt.ylabel('Amount')
plt.xlabel('Cluster')
plt.grid(True)
plt.show()

In [ ]:
# ============================
#     رسم PCA ثلاثي الأبعاد
# ============================

from mpl_toolkits.mplot3d import Axes3D

pca_3d = PCA(n_components=3)
pca_data_3d = pca_3d.fit_transform(df_scaled)

df['Cluster_DBSCAN_3d']= dbscan_labels



fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pca_data_3d[:, 0], pca_data_3d[:, 1], pca_data_3d[:, 2], c=df['Cluster'], cmap='Set1', s=15)
ax.set_title('3D PCA Visualization of DBSCAN Clusters')
plt.show()


# ============================
#   تقييم DBSCAN_3ي (Silhouette)
# ============================

mask = df['Cluster_DBSCAN_3d'] != -1
score = silhouette_score(df_scaled[mask], df['Cluster_DBSCAN_3d'][mask])
print(f"Silhouette Score for DBSCAN_3d: {score:.3f}")



# ============================
#           التباين
# ============================

explained_var_3d = pca_3d.explained_variance_ratio_
display(f"Explained Variance Ratio: {np.round(explained_var_3d,2)}")
display(f"cumulative_variance: {np.round(np.cumsum(explained_var_3d),2)}")
display(f"Total Variance Retained: {np.round(np.sum(explained_var_3d)*100,2)}%")

In [ ]:
# ============================
#      بيانات كل مجموعة
# ============================

cluster_summary_3d = df.groupby('Cluster_DBSCAN_3d').mean()
display(cluster_summary_3d)


In [ ]:
# ============================
#     الرسم البياني للنتائج
# ============================

clustered_df = df.copy()
cluster_means = clustered_df.groupby('Cluster_DBSCAN_3d').mean()

cluster_means[['BALANCE', 'PURCHASES', 'PAYMENTS']].plot(kind='bar', figsize=(10, 6))
plt.title('Average Behavior per Cluster')
plt.ylabel('Amount')
plt.xlabel('Cluster')
plt.grid(True)
plt.show()

## 9. التحليل العنقودي الهرمي (Agglomerative)
- تطبيق خوارزمية Agglomerative Clustering
- تصور النتائج وتقييمها
- مقارنة متوسطات القيم لكل عنقود
- إنشاء رسوم بيانية للسلوك العام لكل مجموعة

In [ ]:
# ============================
#   Agglomerative Clustering
# ============================

from sklearn.cluster import AgglomerativeClustering


agglo = AgglomerativeClustering(n_clusters=K)

agglo_labels = agglo.fit_predict(df_scaled)

df['agglo_labels'] = agglo_labels


#
pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_scaled)


plt.figure(figsize=(8,6))
plt.scatter(pca_data[:, 0], pca_data[:, 1],c=df['agglo_labels'], cmap='Set1', s=15)
plt.title('Agglomerative Clustering (PCA)')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.grid(True)
plt.show()


# ============================
#   تقييم Agglomerative (Silhouette)
# ============================

from sklearn.metrics import silhouette_score

score_agglo = silhouette_score(df_scaled, agglo_labels)
print(f'Silhouette Score for Agglomerative: {score_agglo:.3f}')

# ============================
#          التباين
# ============================

explained_var = pca.explained_variance_ratio_
display(f"Explained Variance Ratio: {np.round(explained_var,2),}")
display(f"cumulative_variance: {np.round(np.cumsum(explained_var),2),}")
display(f"Total Variance Retained: {np.round(np.sum(explained_var)*100,2)}%")

In [ ]:
# ============================
#       بيانات كل مجموعة
# ============================
cluster_summary = df.groupby('agglo_labels').mean()
display(cluster_summary)

In [ ]:
# ============================
#     الرسم البياني للنتائج
# ============================

clustered_df = df.copy()
cluster_means = clustered_df.groupby('agglo_labels').mean()

cluster_means[['BALANCE', 'PURCHASES', 'PAYMENTS']].plot(kind='bar', figsize=(10, 6))
plt.title('Average Behavior per Cluster')
plt.ylabel('Amount')
plt.xlabel('Cluster')
plt.grid(True)
plt.show()